In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import cv2
import copy
dir_project = os.path.join(os.getcwd(), 'drive', 'MyDrive', '研二上 Computer Vision (COMP90086_2021_SM2) 研究生第三学期', 'Final Project')

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from pathlib import Path
import pandas as pd

print(cv2.__version__)

4.1.2


In [ ]:
#code from https://blog.csdn.net/xiaobiyin9140/article/details/84638260

train_img_dir = os.path.join(dir_project, 'train')
# train_img_dir = os.path.join(train_img_dir, 'img')


def read_directory(directory_name):
  array_of_img=[]  
  for filename in os.listdir(r"/"+directory_name):
    img = cv2.imread(directory_name + "/" + filename)
    array_of_img.append(img)
    #print(array_of_img)
  return array_of_img

train_data = read_directory(train_img_dir)

In [ ]:
#This line might result in web crash
# train_data_deepcopy = copy.deepcopy(train_data)

In [ ]:
print(train_data_deepcopy)

[None, None]


In [ ]:
#test_data_deepcopy = copy.deepcopy(test_data)

print(len(test_data))


In [ ]:
from google.colab import drive
import os
import cv2
import numpy as np
from matplotlib import pyplot as plt
from pathlib import Path
import pandas as pd
import platform
from operator import itemgetter 
import time
import platform


In [ ]:
# be sure to mount the directory, or it will fail.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# to tackle the inconsistency between local and colab
prefix = {
    'local' : Path(os.getcwd()).parent,
    'colab' : Path(os.path.join(os.getcwd(), 'drive', 'MyDrive', 'semester4', 'COMP90086', 'assignment', 'final_project'))
}

prepare the directory

In [ ]:
dir_project = prefix['local'] if 'Desktop' in platform.node() else prefix['colab']


label_img_dir = Path(os.path.join(dir_project, 'train', 'img'))
unlabel_img_dir = Path(os.path.join(dir_project, 'test', 'img'))

## Convert the csv file to data frame

In [ ]:
from sklearn.preprocessing import LabelEncoder

root_path = label_img_dir.parent # the grader can define your own root_path to load 'train.csv'

records = pd.read_csv(os.path.join(root_path, 'train.csv')) # load csv file to be a data frame records

# extract file names so that we can easily load images based on this list
fnames = list(records['id'])

# make the goelocation a label. The format is 'x_y'
geoloc = [ f'{x}_{y}' for (x,y) in zip(records['x'],records['y'])]

# build a new data frame df with columns 'fname', 'geoloc', and 'geoloc-label'
df = pd.DataFrame(list(zip(fnames, geoloc)), columns=['fname', 'geoloc']) 

# we will use this line to build a new dictionary called "label2geoloc"
labelencoder = LabelEncoder()
df['geoloc-label'] = labelencoder.fit_transform(df['geoloc']) 



## some collections that can help us process data

In [ ]:
# 1. label2geoloc : this dic can convert label-encoded index to the label
label2geoloc = dict( (label,loc) for loc,label in zip(df['geoloc'], df['geoloc-label']))

# 2. m : we can this dictionary to get label-encoded index of the label by giving filename
m = {}
for (fname, geoloc) in zip(df['fname'], df['geoloc-label']):
    key = geoloc
    if key not in m:
        m[key] = [fname]
    else:
        m[key].append(fname)

## Load the labelled images 

In [ ]:

# input will be the img_dir, the established data frame, and optional args
# output will be a tuple object containing 2 numpy arrays

def gen_img_set(img_dir, df, **kwargs):
    imgs = dict()
    img_objs = []
    img_labels = []
    for (fname, geoloc) in zip(df['fname'], df['geoloc-label']):
        img = cv2.imread(filename = os.path.join(img_dir, f'{fname}.jpg'))
        if len(kwargs) > 0:
            h, w = kwargs['new_dim']
        img = cv2.resize(img, (w, h), interpolation = cv2.INTER_AREA)
        
        img_objs.append(img)
        img_labels.append(geoloc)
        return np.array(img_objs), np.array(img_labels)


In [ ]:
# a use case for the above function
r = 0.3
h, w = (490, 680) # the grader can decide by self.
h, w = int(r*h), int(r*w)

(train_imgs, train_labels) = gen_img_set(label_img_dir, df, new_dim=(h,w))

## The following will be 4 cases:


1.   use CIFAR-10 to train GoogLeNet for initial evaluation
2.   train model with Googlenet and images for this project
3.   transfer learning with VGG19
4.   calculate similarity by VGG19 

## 1. use CIFAR-10 to train GoogLeNet for initial evaluation

In [ ]:
# load all modules
import os
import cv2
import numpy as np
from matplotlib import pyplot as plt
from pathlib import Path
import pandas as pd
import time
import platform
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D


# prepare dimension for image
n = 7500
ratio = 0.8
dim = 224

num_model = 5
# cross-validation, expected to train 5 models
for i in range(num_model):
    # prepare data
    ## download data from the inter-network
    (X_train, y_train), (X_test, y_test) = keras.datasets.cifar10.load_data()[0:n]

    ## prepare training data and label
    train_imgs = np.zeros([int(ratio*n),dim,dim,3], dtype='uint8')
    train_labels = y_train[0:int(ratio*n)]

    ## prepare testing data and label
    test_imgs = np.zeros([int(round((1-ratio)*n,0)) ,dim,dim,3], dtype='uint8')
    test_labels = y_test[0:int(round((1-ratio)*n,0))]

    ## resize input image for training model to 224x224
    for i, img in enumerate(X_train[0:int(ratio*n)]): # (224,224,3)-> (3,224,224)
        large_img = cv2.resize(img, dsize=(dim, dim), interpolation=cv2.INTER_CUBIC)
        train_imgs[i] = large_img

    ## resize input image for testing model to 224x224
    for i, img in enumerate(X_test[0:int(round((1-ratio)*n,0))]): # (224,224,3)-> (3,224,224)
        large_img = cv2.resize(img, dsize=(dim, dim), interpolation=cv2.INTER_CUBIC)
        test_imgs[i] = large_img



    # prepare model
    ## create the base pre-trained model
    base_model = InceptionV3(weights='imagenet', include_top=False)
    ## add a global spatial average pooling layer
    x = base_model.output
    x = GlobalAveragePooling2D()(x)

    ## let's add a fully-connected layer
    x = Dense(1024, activation='relu')(x)

    ## and a logistic layer -- let's say we have 200 classes
    predictions = Dense(1499, activation='softmax')(x)
    ## this is the model we will train
    model = Model(inputs=base_model.input, outputs=predictions)
    ## compile model
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
               loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False), 
               metrics=['accuracy'])

    ## train the model
    # train and verification
    model_history = model.fit(train_imgs, train_labels, 
                          epochs=10, batch_size=32, 
                          validation_data=(test_imgs, test_labels))

## 2. Train model with GoogLeNet


In [ ]:
# with cross-validation
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

# 1.prepare data first
h, w = (224,224)
(train_imgs, train_labels) = gen_img_set(train_img_dir, df, new_dim=(h,w))



# 2. prepare data, sampling from the original to create 3 different sets
from sklearn.model_selection import KFold
seed = 7
np.random.seed(seed)
kfold = KFold(n_splits=3, shuffle=True, random_state=1)


X = model_imgs
Y = model_labels

# 3. prepare model GoogLeNet
base_model = InceptionV3(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1499, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)


# 4. train model. This will train 3 distinct models
googlenet_model_histories = []  # record all trained models
for i, (train, test) in enumerate(kfold.split(X, Y)):
    print(f'i={i}')
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
           loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False), #because we take a coordinate as an independent label.
           metrics=['accuracy'])
    model_history = model.fit(X[train], Y[train], epochs=10, batch_size=10, validation_data=(X[test], Y[test]))
    
    y_true = np.array(Y[test])
    y_pred = gen_pred_labels(model.predict(X[test]))
    error = compute_err(y_pred, y_true, label2geoloc)
    print(error)
    
    googlenet_model_histories.append(model_history)

## Transfer learning with VGG19

In [ ]:
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.applications.vgg19 import VGG19

# 1.prepare data first
r = 0.3
h, w = list(imgs_raw.values())[0].shape[:2]
h, w = int(r*h), int(r*w)

(train_imgs, train_labels) = gen_img_set(label_img_dir, df, new_dim=(h,w))


# 2. prepare data, sampling from the original to create 5 different sets
seed = 7
np.random.seed(seed)
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)


X = train_imgs
Y = train_labels

# 3.prepare model
base_model = VGG19(weights='imagenet', input_shape=list(X[0].shape),include_top=False) #set the input size to be the size of the training testing images
fea_model= tf.keras.Model(base_model.inputs, base_model.layers[-1].output)


# 4.train model. This will build 5 distinct models.
for i, (train, test) in enumerate(kfold.split(X, Y)):
    print(i, train.shape, test.shape)
  
    ss_train_fea=fea_model.predict(X[train])
    ss_test_fea=fea_model.predict(X[test])
    visible = layers.Input(shape=ss_train_fea.shape[1:])
    
    flat = layers.Flatten()(visible)
    hidden = layers.Dense(units=1024, activation='relu')(flat)
    hidden = layers.Dropout(0.5)(hidden)
    output = layers.Dense(units=1499, activation='softmax')(hidden)

    model = Model(inputs=visible, outputs=output)

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
           loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
           metrics=['accuracy']) 

    history_model = model.fit(ss_train_fea, np.array(Y[train]), epochs=20, batch_size=32, 
                              validation_data=(ss_test_fea, np.array(Y[test])))
    
    score = model.evaluate(ss_test_fea, np.array(Y[test]))


## Zero Rule

In [ ]:
# Read File
data_train_full = pd.read_csv('train.csv', sep=',')

y_train_full = data_train_full['x']

x_train_full = data_train_full['x']

In [ ]:
from collections import Counter
y_zero_r_train = Counter(y_train_full).most_common()
y_majority_class = y_zero_r_train[0][0]

x_zero_r_train = Counter(x_train_full).most_common()
x_majority_class = x_zero_r_train[0][0]

## The feature matching function
The code of this function are from Tutorial(week 7).
We might need to add some future extraction function to this code.

In [ ]:
def compare(gray,scene_gray):
  sift = cv2.SIFT_create() # if cv2 version >= 4.4.0 
  # sift = cv2.xfeatures2d.SIFT_create() # if cv2 version = 4.3.x 
  
  # Compute SIFT keypoints and descriptors
  kp1, des1 = sift.detectAndCompute(gray,None)
  kp2, des2 = sift.detectAndCompute(scene_gray,None)

  # if kp1 == [] or len(kp1)<5 or kp2 == [] or len(kp2) < 5:
    # return 0

  FLANN_INDEX_KDTREE = 1
  index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
  search_params = dict(checks=50)   # or pass empty dictionary
  flann = cv2.FlannBasedMatcher(index_params,search_params)

  # Matching descriptor using KNN algorithm
  matches = flann.knnMatch(des1,des2,k=2)

  # Create a mask to draw all good matches
  matchesMask = []

  # Store all good matches as per Lowe's Ratio test.
  # good = []
  good=0
  for m,n in matches:
    if m.distance < 0.7*n.distance:
        # good.append(m)
        good = good + 1
        matchesMask.append([1,0]) # Match
    else:
        matchesMask.append([0,0]) # Mismatch
       
        

  # good_matches = cv2.drawMatchesKnn(gray,kp1,scene_gray,kp2,matches,None,**draw_params)

  return (good/len(matchesMask))

In [ ]:
result_name = []

for j in test_data_name_list:
    k = 0
    for i in train_data_name_list:
        try:
            current_result=compare(train_data[i],test_data[j])
        except:
            continue
        if current_result > k:
            k = current_result
            current_name = i
        else:
            continue
    if k == 0:
        current_name = choice(train_data_name_list)
    print(current_name)
    result_name.append(current_name)


print('result_name:',result_name)

In [ ]:
# It tooks me two days to run this code
result_names = ['IMG3628_5.jpg', 'IMG4145_4.jpg', 'IMG2757_1.jpg', 'IMG3323_2.jpg', 'IMG3232_4.jpg', 'IMG2792_3.jpg', 'IMG3762_5.jpg', 'IMG3709_3.jpg', 'IMG2768_2.jpg', 'IMG2923_4.jpg', 'IMG3662_2.jpg', 'IMG3104_4.jpg', 'IMG3323_2.jpg', 'IMG3323_2.jpg', 'IMG3286_4.jpg', 'IMG3036_5.jpg', 'IMG3368_2.jpg', 'IMG3950_4.jpg', 'IMG3645_4.jpg', 'IMG3323_2.jpg', 'IMG2792_3.jpg', 'IMG3323_2.jpg', 'IMG2867_2.jpg', 'IMG3232_4.jpg', 'IMG3725_1.jpg', 'IMG3036_5.jpg', 'IMG3171_2.jpg', 'IMG3845_5.jpg', 'IMG2804_2.jpg', 'IMG3232_4.jpg', 'IMG3323_2.jpg', 'IMG2883_1.jpg', 'IMG2867_2.jpg', 'IMG3845_5.jpg', 'IMG3232_4.jpg', 'IMG3323_2.jpg', 'IMG3575_3.jpg', 'IMG3323_2.jpg', 'IMG2772_2.jpg', 'IMG3104_2.jpg', 'IMG3382_2.jpg', 'IMG3232_4.jpg', 'IMG3323_2.jpg', 'IMG3229_5.jpg', 'IMG3794_2.jpg', 'IMG3845_5.jpg', 'IMG3323_2.jpg', 'IMG3368_2.jpg', 'IMG2845_4.jpg', 'IMG4188_1.jpg', 'IMG3323_2.jpg', 'IMG2983_2.jpg', 'IMG3880_4.jpg', 'IMG2792_3.jpg', 'IMG3721_4.jpg', 'IMG3334_2.jpg', 'IMG3753_3.jpg', 'IMG3283_4.jpg', 'IMG2800_1.jpg', 'IMG3762_5.jpg', 'IMG3709_3.jpg', 'IMG3953_2.jpg', 'IMG2817_5.jpg', 'IMG3848_3.jpg', 'IMG3502_2.jpg', 'IMG2768_2.jpg', 'IMG3343_2.jpg', 'IMG3283_4.jpg', 'IMG3323_2.jpg', 'IMG3334_2.jpg', 'IMG2867_2.jpg', 'IMG3502_3.jpg', 'IMG3993_2.jpg', 'IMG3102_4.jpg', 'IMG2867_2.jpg', 'IMG3323_2.jpg', 'IMG3323_2.jpg', 'IMG3762_5.jpg', 'IMG3992_2.jpg', 'IMG3762_5.jpg', 'IMG3323_2.jpg', 'IMG3232_4.jpg', 'IMG4145_4.jpg', 'IMG4031_5.jpg', 'IMG3420_1.jpg', 'IMG3415_1.jpg', 'IMG4145_4.jpg', 'IMG3461_2.jpg', 'IMG2907_3.jpg', 'IMG3762_5.jpg', 'IMG3323_2.jpg', 'IMG2845_4.jpg', 'IMG2746_4.jpg', 'IMG3953_2.jpg', 'IMG3149_3.jpg', 'IMG3323_2.jpg', 'IMG3874_4.jpg', 'IMG3323_2.jpg', 'IMG3794_2.jpg', 'IMG3323_2.jpg', 'IMG3323_2.jpg', 'IMG3200_4.jpg', 'IMG3368_2.jpg', 'IMG3603_1.jpg', 'IMG3323_2.jpg', 'IMG3036_5.jpg', 'IMG3028_4.jpg', 'IMG3323_2.jpg', 'IMG3232_4.jpg', 'IMG2867_2.jpg', 'IMG3382_2.jpg', 'IMG3805_4.jpg', 'IMG3102_4.jpg', 'IMG3036_5.jpg', 'IMG3323_2.jpg', 'IMG3323_2.jpg', 'IMG3323_2.jpg', 'IMG3794_2.jpg', 'IMG4080_3.jpg', 'IMG3502_3.jpg', 'IMG3189_3.jpg', 'IMG3072_2.jpg', 'IMG3323_2.jpg', 'IMG3164_3.jpg', 'IMG3036_5.jpg', 'IMG4196_3.jpg', 'IMG3953_2.jpg', 'IMG3867_5.jpg', 'IMG3675_4.jpg', 'IMG3880_4.jpg', 'IMG4025_1.jpg', 'IMG3794_2.jpg', 'IMG4013_1.jpg', 'IMG2772_2.jpg', 'IMG3232_4.jpg', 'IMG4183_4.jpg', 'IMG3554_2.jpg', 'IMG3232_4.jpg', 'IMG3603_5.jpg', 'IMG2867_2.jpg', 'IMG4150_2.jpg', 'IMG2772_2.jpg', 'IMG2792_3.jpg', 'IMG3848_3.jpg', 'IMG2800_5.jpg', 'IMG3794_2.jpg', 'IMG2746_4.jpg', 'IMG3828_2.jpg', 'IMG3867_5.jpg', 'IMG2867_2.jpg', 'IMG2792_3.jpg', 'IMG3232_4.jpg', 'IMG2907_3.jpg', 'IMG2792_3.jpg', 'IMG3323_2.jpg', 'IMG3540_2.jpg', 'IMG3323_2.jpg', 'IMG3435_3.jpg', 'IMG3229_5.jpg', 'IMG4126_4.jpg', 'IMG3232_4.jpg', 'IMG3323_2.jpg', 'IMG4168_1.jpg', 'IMG3950_4.jpg', 'IMG3232_4.jpg', 'IMG3120_1.jpg', 'IMG4013_1.jpg', 'IMG3368_2.jpg', 'IMG2784_2.jpg', 'IMG2863_2.jpg', 'IMG3582_4.jpg', 'IMG3448_5.jpg', 'IMG3953_2.jpg', 'IMG3036_5.jpg', 'IMG3120_1.jpg', 'IMG3762_5.jpg', 'IMG2863_2.jpg', 'IMG2746_2.jpg', 'IMG3415_5.jpg', 'IMG3368_2.jpg', 'IMG3368_2.jpg', 'IMG3766_3.jpg', 'IMG3154_5.jpg', 'IMG3323_2.jpg', 'IMG3323_2.jpg', 'IMG3368_2.jpg', 'IMG3036_5.jpg', 'IMG3383_1.jpg', 'IMG3036_5.jpg', 'IMG2867_2.jpg', 'IMG3311_2.jpg', 'IMG4141_4.jpg', 'IMG3323_2.jpg', 'IMG3232_4.jpg', 'IMG3229_5.jpg', 'IMG3189_3.jpg', 'IMG3036_5.jpg', 'IMG3848_3.jpg', 'IMG3323_2.jpg', 'IMG3104_2.jpg', 'IMG4188_1.jpg', 'IMG3334_2.jpg', 'IMG3500_2.jpg', 'IMG3435_3.jpg', 'IMG3426_1.jpg', 'IMG3901_5.jpg', 'IMG3323_2.jpg', 'IMG3709_3.jpg', 'IMG3229_5.jpg', 'IMG2867_2.jpg', 'IMG3323_2.jpg', 'IMG3845_5.jpg', 'IMG3232_4.jpg', 'IMG3179_5.jpg', 'IMG4130_1.jpg', 'IMG3232_4.jpg', 'IMG3232_4.jpg', 'IMG3232_4.jpg', 'IMG2772_2.jpg', 'IMG3794_2.jpg', 'IMG3448_5.jpg', 'IMG4149_3.jpg', 'IMG3762_5.jpg', 'IMG3232_4.jpg', 'IMG3794_2.jpg', 'IMG2800_1.jpg', 'IMG4188_1.jpg', 'IMG3880_4.jpg', 'IMG2867_2.jpg', 'IMG3334_2.jpg', 'IMG3232_4.jpg', 'IMG3762_5.jpg', 'IMG3415_1.jpg', 'IMG4067_5.jpg', 'IMG4145_4.jpg', 'IMG3435_3.jpg', 'IMG3896_1.jpg', 'IMG3357_4.jpg', 'IMG2867_2.jpg', 'IMG3036_5.jpg', 'IMG3323_2.jpg', 'IMG3229_5.jpg', 'IMG2845_4.jpg', 'IMG3323_2.jpg', 'IMG3794_2.jpg', 'IMG3036_5.jpg', 'IMG3036_5.jpg', 'IMG2914_2.jpg', 'IMG3582_4.jpg', 'IMG3323_2.jpg', 'IMG3323_2.jpg', 'IMG3368_2.jpg', 'IMG3934_1.jpg', 'IMG3575_3.jpg', 'IMG3323_2.jpg', 'IMG3232_4.jpg', 'IMG3848_3.jpg', 'IMG3323_2.jpg', 'IMG2768_2.jpg', 'IMG3334_2.jpg', 'IMG2854_5.jpg', 'IMG3334_2.jpg', 'IMG2746_2.jpg', 'IMG3200_4.jpg', 'IMG3368_2.jpg', 'IMG3323_2.jpg', 'IMG3368_2.jpg', 'IMG3036_5.jpg', 'IMG3621_2.jpg', 'IMG3874_4.jpg', 'IMG3164_3.jpg', 'IMG3323_2.jpg', 'IMG3794_2.jpg', 'IMG4145_4.jpg', 'IMG3104_4.jpg', 'IMG3036_5.jpg', 'IMG2929_1.jpg', 'IMG3603_5.jpg', 'IMG2867_2.jpg', 'IMG3323_2.jpg', 'IMG3950_4.jpg', 'IMG3686_4.jpg', 'IMG3323_2.jpg', 'IMG3200_4.jpg', 'IMG2792_3.jpg', 'IMG2768_2.jpg', 'IMG3323_2.jpg', 'IMG3725_1.jpg', 'IMG2803_5.jpg', 'IMG3200_4.jpg', 'IMG2887_4.jpg', 'IMG3368_2.jpg', 'IMG3323_2.jpg', 'IMG3334_2.jpg', 'IMG2768_2.jpg', 'IMG2816_2.jpg', 'IMG3603_5.jpg', 'IMG2746_2.jpg', 'IMG3845_5.jpg', 'IMG3357_4.jpg', 'IMG3368_2.jpg', 'IMG4226_3.jpg', 'IMG3334_2.jpg', 'IMG3845_5.jpg', 'IMG3196_3.jpg', 'IMG4001_4.jpg', 'IMG3323_2.jpg', 'IMG3323_2.jpg', 'IMG3885_4.jpg', 'IMG3323_2.jpg', 'IMG3149_3.jpg', 'IMG3357_4.jpg', 'IMG3232_4.jpg', 'IMG3323_2.jpg', 'IMG3379_1.jpg', 'IMG3747_4.jpg', 'IMG3451_5.jpg', 'IMG3104_2.jpg', 'IMG3368_2.jpg', 'IMG4145_4.jpg', 'IMG3950_4.jpg', 'IMG3323_2.jpg', 'IMG2867_2.jpg', 'IMG3323_2.jpg', 'IMG3323_2.jpg', 'IMG3334_2.jpg', 'IMG3709_3.jpg', 'IMG3323_2.jpg', 'IMG3334_2.jpg', 'IMG3420_1.jpg', 'IMG3334_2.jpg', 'IMG3502_2.jpg', 'IMG4234_5.jpg', 'IMG3575_3.jpg', 'IMG2863_2.jpg', 'IMG3323_2.jpg', 'IMG3603_5.jpg', 'IMG3104_4.jpg', 'IMG2867_2.jpg', 'IMG2792_3.jpg', 'IMG3762_5.jpg', 'IMG2903_4.jpg', 'IMG4196_3.jpg', 'IMG3323_2.jpg', 'IMG3828_2.jpg', 'IMG3229_5.jpg', 'IMG3323_2.jpg', 'IMG3368_2.jpg', 'IMG3323_2.jpg', 'IMG3762_5.jpg', 'IMG3287_5.jpg', 'IMG3334_2.jpg', 'IMG3368_2.jpg', 'IMG3164_1.jpg', 'IMG3323_2.jpg', 'IMG2772_2.jpg', 'IMG2788_4.jpg', 'IMG3334_2.jpg', 'IMG3896_1.jpg', 'IMG3725_1.jpg', 'IMG4107_3.jpg', 'IMG3164_3.jpg', 'IMG2867_2.jpg', 'IMG2772_2.jpg', 'IMG3232_4.jpg', 'IMG3323_2.jpg', 'IMG3368_2.jpg', 'IMG4150_2.jpg', 'IMG3323_2.jpg', 'IMG3845_5.jpg', 'IMG3502_2.jpg', 'IMG2804_2.jpg', 'IMG3323_2.jpg', 'IMG3323_2.jpg', 'IMG3232_4.jpg', 'IMG2883_1.jpg', 'IMG3164_3.jpg', 'IMG3323_2.jpg', 'IMG3692_4.jpg', 'IMG3323_2.jpg', 'IMG2867_2.jpg', 'IMG3323_2.jpg', 'IMG3725_1.jpg', 'IMG4130_1.jpg', 'IMG3036_5.jpg', 'IMG3323_2.jpg', 'IMG3415_1.jpg', 'IMG3171_2.jpg', 'IMG2883_1.jpg', 'IMG3323_2.jpg', 'IMG3845_5.jpg', 'IMG2903_4.jpg', 'IMG3725_1.jpg', 'IMG3323_2.jpg', 'IMG3958_5.jpg', 'IMG3420_1.jpg', 'IMG3575_3.jpg', 'IMG3845_5.jpg', 'IMG3334_2.jpg', 'IMG3077_3.jpg', 'IMG3036_5.jpg', 'IMG3323_2.jpg', 'IMG3334_2.jpg', 'IMG3232_4.jpg', 'IMG3311_2.jpg', 'IMG3420_1.jpg', 'IMG3382_2.jpg', 'IMG3323_2.jpg', 'IMG3645_4.jpg', 'IMG2792_3.jpg', 'IMG3727_2.jpg', 'IMG3232_4.jpg', 'IMG3953_2.jpg', 'IMG2882_2.jpg', 'IMG2863_2.jpg', 'IMG3232_4.jpg', 'IMG3845_5.jpg', 'IMG3323_2.jpg', 'IMG4234_5.jpg', 'IMG3068_3.jpg', 'IMG3334_2.jpg', 'IMG3762_5.jpg', 'IMG3382_2.jpg', 'IMG3992_2.jpg', 'IMG3334_2.jpg', 'IMG3323_2.jpg', 'IMG3334_2.jpg', 'IMG3709_3.jpg', 'IMG3323_2.jpg', 'IMG4150_2.jpg', 'IMG3120_1.jpg', 'IMG3645_4.jpg', 'IMG4130_1.jpg', 'IMG3577_3.jpg', 'IMG2964_3.jpg', 'IMG2768_2.jpg', 'IMG3382_2.jpg', 'IMG3334_2.jpg', 'IMG3323_2.jpg', 'IMG2792_3.jpg', 'IMG3373_3.jpg', 'IMG3323_2.jpg', 'IMG3036_5.jpg', 'IMG3500_2.jpg', 'IMG3323_2.jpg', 'IMG3582_4.jpg', 'IMG3950_4.jpg', 'IMG3200_4.jpg', 'IMG3323_2.jpg', 'IMG3709_3.jpg', 'IMG3323_2.jpg', 'IMG3405_4.jpg', 'IMG3334_2.jpg', 'IMG3323_2.jpg', 'IMG2975_5.jpg', 'IMG2817_5.jpg', 'IMG2792_3.jpg', 'IMG2772_2.jpg', 'IMG3366_1.jpg', 'IMG2782_5.jpg', 'IMG3368_2.jpg', 'IMG4116_5.jpg', 'IMG3762_5.jpg', 'IMG2845_4.jpg', 'IMG3725_1.jpg', 'IMG3992_2.jpg', 'IMG3323_2.jpg', 'IMG3848_3.jpg', 'IMG3243_1.jpg', 'IMG3323_2.jpg', 'IMG3334_2.jpg', 'IMG2867_2.jpg', 'IMG3794_2.jpg', 'IMG3762_5.jpg', 'IMG2768_2.jpg', 'IMG3323_2.jpg', 'IMG2792_3.jpg', 'IMG3603_5.jpg', 'IMG3681_5.jpg', 'IMG3911_4.jpg', 'IMG3323_2.jpg', 'IMG3582_4.jpg', 'IMG3323_2.jpg', 'IMG3036_5.jpg', 'IMG4168_1.jpg', 'IMG3052_3.jpg', 'IMG3794_2.jpg', 'IMG4203_3.jpg', 'IMG3435_3.jpg', 'IMG3468_2.jpg', 'IMG2772_2.jpg', 'IMG3323_2.jpg', 'IMG2927_3.jpg', 'IMG3953_2.jpg', 'IMG4196_3.jpg', 'IMG3323_2.jpg', 'IMG3420_1.jpg', 'IMG3794_2.jpg', 'IMG3848_3.jpg', 'IMG3323_2.jpg', 'IMG2867_2.jpg', 'IMG3848_3.jpg', 'IMG3171_2.jpg', 'IMG4198_3.jpg', 'IMG3334_2.jpg', 'IMG3323_2.jpg', 'IMG4234_5.jpg', 'IMG3229_5.jpg', 'IMG3415_1.jpg', 'IMG3036_5.jpg', 'IMG3709_3.jpg', 'IMG3382_2.jpg', 'IMG2768_2.jpg', 'IMG3753_3.jpg', 'IMG2792_3.jpg', 'IMG3323_2.jpg', 'IMG3283_4.jpg', 'IMG2914_2.jpg', 'IMG4153_1.jpg', 'IMG2802_1.jpg', 'IMG2929_1.jpg', 'IMG2792_3.jpg', 'IMG3323_2.jpg', 'IMG2768_2.jpg', 'IMG3816_4.jpg', 'IMG3766_3.jpg', 'IMG3615_2.jpg', 'IMG3961_4.jpg', 'IMG3232_4.jpg', 'IMG3323_2.jpg', 'IMG3247_2.jpg', 'IMG4150_2.jpg', 'IMG3232_4.jpg', 'IMG3229_5.jpg', 'IMG3232_4.jpg', 'IMG3323_2.jpg', 'IMG3645_4.jpg', 'IMG3323_2.jpg', 'IMG2863_2.jpg', 'IMG3304_3.jpg', 'IMG3334_2.jpg', 'IMG3247_2.jpg', 'IMG3323_2.jpg', 'IMG3709_3.jpg', 'IMG3368_2.jpg', 'IMG3953_2.jpg', 'IMG3323_2.jpg', 'IMG3323_2.jpg', 'IMG3323_2.jpg', 'IMG3719_4.jpg', 'IMG3665_3.jpg', 'IMG3334_2.jpg', 'IMG3068_3.jpg', 'IMG3762_5.jpg', 'IMG3232_4.jpg', 'IMG3794_2.jpg', 'IMG3575_3.jpg', 'IMG2975_5.jpg', 'IMG3323_2.jpg', 'IMG2928_5.jpg', 'IMG3762_5.jpg', 'IMG3738_3.jpg', 'IMG2768_2.jpg', 'IMG3901_5.jpg', 'IMG3171_2.jpg', 'IMG3738_3.jpg', 'IMG3060_5.jpg', 'IMG3334_2.jpg', 'IMG3232_4.jpg', 'IMG3048_1.jpg', 'IMG3323_2.jpg', 'IMG3154_5.jpg', 'IMG3380_2.jpg', 'IMG2879_5.jpg', 'IMG3171_2.jpg', 'IMG3243_1.jpg', 'IMG2923_4.jpg', 'IMG3232_4.jpg', 'IMG4145_4.jpg', 'IMG3247_2.jpg', 'IMG3845_5.jpg', 'IMG3368_2.jpg', 'IMG3323_2.jpg', 'IMG3229_5.jpg', 'IMG3247_2.jpg', 'IMG3120_1.jpg', 'IMG3164_3.jpg', 'IMG4226_3.jpg', 'IMG3323_2.jpg', 'IMG3575_3.jpg', 'IMG3323_2.jpg', 'IMG3575_3.jpg', 'IMG2927_3.jpg', 'IMG3317_1.jpg', 'IMG3323_2.jpg', 'IMG3171_2.jpg', 'IMG3540_2.jpg', 'IMG3773_1.jpg', 'IMG2845_4.jpg', 'IMG4142_5.jpg', 'IMG3323_2.jpg', 'IMG3368_2.jpg', 'IMG4107_3.jpg', 'IMG3845_5.jpg', 'IMG2867_2.jpg', 'IMG3066_3.jpg', 'IMG2784_2.jpg', 'IMG3323_2.jpg', 'IMG3323_2.jpg', 'IMG2888_2.jpg', 'IMG3845_5.jpg', 'IMG3120_1.jpg', 'IMG3036_5.jpg', 'IMG3232_4.jpg', 'IMG2923_4.jpg', 'IMG3334_2.jpg', 'IMG3323_2.jpg', 'IMG3229_5.jpg', 'IMG4025_1.jpg', 'IMG3334_2.jpg', 'IMG3323_2.jpg', 'IMG3368_2.jpg', 'IMG3104_2.jpg', 'IMG3323_2.jpg', 'IMG3934_1.jpg', 'IMG3880_4.jpg', 'IMG3794_2.jpg', 'IMG4145_4.jpg', 'IMG3625_2.jpg', 'IMG3368_2.jpg', 'IMG3232_4.jpg', 'IMG3323_2.jpg', 'IMG2928_5.jpg', 'IMG2964_3.jpg', 'IMG3323_2.jpg', 'IMG3323_2.jpg', 'IMG3368_2.jpg', 'IMG3232_4.jpg', 'IMG3311_2.jpg', 'IMG3828_2.jpg', 'IMG2902_1.jpg', 'IMG3368_2.jpg', 'IMG3334_2.jpg', 'IMG3164_3.jpg', 'IMG3386_2.jpg', 'IMG3366_1.jpg', 'IMG3323_2.jpg', 'IMG3232_4.jpg', 'IMG2867_2.jpg', 'IMG2902_1.jpg', 'IMG3154_5.jpg', 'IMG3323_2.jpg', 'IMG3036_5.jpg', 'IMG3762_5.jpg', 'IMG3036_5.jpg', 'IMG3556_2.jpg', 'IMG3874_4.jpg', 'IMG4053_1.jpg', 'IMG2867_2.jpg', 'IMG3845_5.jpg', 'IMG4145_4.jpg', 'IMG3597_2.jpg', 'IMG3794_2.jpg', 'IMG2975_5.jpg', 'IMG2786_4.jpg', 'IMG3323_2.jpg', 'IMG3232_4.jpg', 'IMG3323_2.jpg', 'IMG3404_2.jpg', 'IMG3323_2.jpg', 'IMG3582_4.jpg', 'IMG3323_2.jpg', 'IMG3323_2.jpg', 'IMG3323_2.jpg', 'IMG2867_2.jpg', 'IMG4025_1.jpg', 'IMG3323_2.jpg', 'IMG3794_2.jpg', 'IMG3762_5.jpg', 'IMG2928_5.jpg', 'IMG3323_2.jpg', 'IMG4145_4.jpg', 'IMG3999_3.jpg', 'IMG3323_2.jpg', 'IMG3540_2.jpg', 'IMG3415_1.jpg', 'IMG3229_5.jpg', 'IMG3323_2.jpg', 'IMG3368_2.jpg', 'IMG3848_3.jpg', 'IMG3232_4.jpg', 'IMG3738_3.jpg', 'IMG3323_2.jpg', 'IMG3603_5.jpg', 'IMG3036_5.jpg', 'IMG3171_2.jpg', 'IMG3077_3.jpg', 'IMG3575_3.jpg', 'IMG3738_3.jpg', 'IMG2929_1.jpg', 'IMG3036_5.jpg', 'IMG3845_5.jpg', 'IMG3334_2.jpg', 'IMG2867_2.jpg', 'IMG3950_4.jpg', 'IMG3323_2.jpg', 'IMG2792_3.jpg', 'IMG4025_1.jpg', 'IMG4137_2.jpg', 'IMG3323_2.jpg', 'IMG4130_1.jpg', 'IMG3575_3.jpg', 'IMG3323_2.jpg', 'IMG2768_2.jpg', 'IMG3164_1.jpg', 'IMG3120_1.jpg', 'IMG2902_1.jpg', 'IMG3229_5.jpg', 'IMG3323_2.jpg', 'IMG3248_1.jpg', 'IMG3323_2.jpg', 'IMG3247_2.jpg', 'IMG3323_2.jpg', 'IMG2768_2.jpg', 'IMG4141_4.jpg', 'IMG3323_2.jpg', 'IMG3323_2.jpg', 'IMG3057_3.jpg', 'IMG3036_5.jpg', 'IMG2792_3.jpg', 'IMG3334_2.jpg', 'IMG3323_2.jpg', 'IMG3323_2.jpg', 'IMG3104_2.jpg', 'IMG2867_2.jpg', 'IMG3334_2.jpg', 'IMG2867_2.jpg', 'IMG2919_4.jpg', 'IMG3334_2.jpg', 'IMG3323_2.jpg', 'IMG3248_1.jpg', 'IMG3129_5.jpg', 'IMG4198_3.jpg', 'IMG3415_1.jpg', 'IMG3283_4.jpg', 'IMG3762_5.jpg', 'IMG3323_2.jpg', 'IMG4150_2.jpg', 'IMG3762_5.jpg', 'IMG3334_2.jpg', 'IMG3232_4.jpg', 'IMG3709_3.jpg', 'IMG3794_2.jpg', 'IMG4234_5.jpg', 'IMG3575_3.jpg', 'IMG3232_4.jpg', 'IMG3334_2.jpg', 'IMG3368_2.jpg', 'IMG3028_4.jpg', 'IMG2883_1.jpg', 'IMG3323_2.jpg', 'IMG3500_2.jpg', 'IMG3215_1.jpg', 'IMG4150_2.jpg', 'IMG3368_2.jpg', 'IMG3415_1.jpg', 'IMG3794_2.jpg', 'IMG3317_1.jpg', 'IMG3070_2.jpg', 'IMG3794_2.jpg', 'IMG3232_4.jpg', 'IMG2792_3.jpg', 'IMG2792_3.jpg', 'IMG2772_2.jpg', 'IMG2768_2.jpg', 'IMG2772_2.jpg', 'IMG3848_3.jpg', 'IMG3893_3.jpg', 'IMG3048_1.jpg', 'IMG3036_5.jpg', 'IMG3645_4.jpg', 'IMG3323_2.jpg', 'IMG3603_5.jpg', 'IMG3154_5.jpg', 'IMG3323_2.jpg', 'IMG3738_3.jpg', 'IMG3334_2.jpg', 'IMG3334_2.jpg', 'IMG3028_1.jpg', 'IMG3334_2.jpg', 'IMG3845_5.jpg', 'IMG3848_3.jpg', 'IMG3251_3.jpg', 'IMG2804_2.jpg', 'IMG3028_1.jpg', 'IMG3323_2.jpg', 'IMG3762_5.jpg', 'IMG3323_2.jpg', 'IMG2894_1.jpg', 'IMG3036_5.jpg', 'IMG3603_5.jpg', 'IMG3323_2.jpg', 'IMG3036_5.jpg', 'IMG3280_1.jpg', 'IMG3502_2.jpg', 'IMG3088_3.jpg', 'IMG4196_3.jpg', 'IMG3794_2.jpg', 'IMG3323_2.jpg', 'IMG3415_1.jpg', 'IMG4203_3.jpg', 'IMG3645_4.jpg', 'IMG3323_2.jpg', 'IMG3468_2.jpg', 'IMG3950_4.jpg', 'IMG3323_2.jpg', 'IMG3794_2.jpg', 'IMG3953_2.jpg', 'IMG3232_4.jpg', 'IMG3911_4.jpg', 'IMG3725_1.jpg', 'IMG2923_4.jpg', 'IMG3845_5.jpg', 'IMG3323_2.jpg', 'IMG3323_2.jpg', 'IMG3334_2.jpg', 'IMG3171_2.jpg', 'IMG3232_4.jpg', 'IMG3368_2.jpg', 'IMG4183_4.jpg', 'IMG4150_2.jpg', 'IMG3323_2.jpg', 'IMG3334_2.jpg', 'IMG3323_2.jpg', 'IMG3725_1.jpg', 'IMG2803_5.jpg', 'IMG3334_2.jpg', 'IMG3549_4.jpg', 'IMG3794_2.jpg', 'IMG2768_2.jpg', 'IMG4149_5.jpg', 'IMG4141_4.jpg', 'IMG3645_4.jpg', 'IMG2802_1.jpg', 'IMG2975_5.jpg', 'IMG3794_2.jpg', 'IMG3738_3.jpg', 'IMG3323_2.jpg', 'IMG2914_2.jpg', 'IMG4186_4.jpg', 'IMG3224_2.jpg', 'IMG3415_1.jpg', 'IMG3588_1.jpg', 'IMG3621_2.jpg', 'IMG3248_1.jpg', 'IMG3961_4.jpg', 'IMG3323_2.jpg', 'IMG3036_5.jpg', 'IMG3164_3.jpg', 'IMG4150_2.jpg', 'IMG3323_2.jpg', 'IMG3845_5.jpg', 'IMG3575_3.jpg', 'IMG3615_2.jpg', 'IMG2792_3.jpg', 'IMG3435_3.jpg', 'IMG3323_2.jpg', 'IMG3323_2.jpg', 'IMG3334_2.jpg', 'IMG3958_5.jpg', 'IMG3311_2.jpg', 'IMG3247_2.jpg', 'IMG3334_2.jpg', 'IMG3334_2.jpg', 'IMG3323_2.jpg', 'IMG3232_4.jpg', 'IMG4013_1.jpg', 'IMG4109_4.jpg', 'IMG3308_4.jpg', 'IMG3232_4.jpg', 'IMG4007_4.jpg', 'IMG3068_3.jpg', 'IMG4150_2.jpg', 'IMG4150_2.jpg', 'IMG3323_2.jpg', 'IMG3958_5.jpg', 'IMG2802_1.jpg', 'IMG3323_2.jpg', 'IMG3334_2.jpg', 'IMG3725_1.jpg', 'IMG2792_3.jpg', 'IMG2792_3.jpg', 'IMG3368_2.jpg', 'IMG3762_5.jpg', 'IMG4013_1.jpg', 'IMG3763_3.jpg', 'IMG3323_2.jpg', 'IMG3847_2.jpg', 'IMG3323_2.jpg', 'IMG3323_2.jpg', 'IMG3415_1.jpg', 'IMG3848_3.jpg', 'IMG2800_1.jpg', 'IMG3323_2.jpg', 'IMG3845_5.jpg', 'IMG3141_4.jpg', 'IMG2792_3.jpg', 'IMG3323_2.jpg', 'IMG3232_4.jpg', 'IMG3028_1.jpg', 'IMG4168_1.jpg', 'IMG3334_2.jpg', 'IMG3154_5.jpg', 'IMG3958_5.jpg', 'IMG3373_3.jpg', 'IMG2867_2.jpg', 'IMG3323_2.jpg', 'IMG2867_2.jpg', 'IMG3813_2.jpg', 'IMG3334_2.jpg', 'IMG3323_2.jpg', 'IMG3709_3.jpg', 'IMG3323_2.jpg', 'IMG3048_1.jpg', 'IMG3435_3.jpg', 'IMG3323_2.jpg', 'IMG3323_2.jpg', 'IMG3323_2.jpg', 'IMG3323_2.jpg', 'IMG3366_1.jpg', 'IMG4234_5.jpg', 'IMG3294_5.jpg', 'IMG3334_2.jpg', 'IMG3597_2.jpg', 'IMG2803_5.jpg', 'IMG3036_5.jpg', 'IMG3575_3.jpg', 'IMG2772_2.jpg', 'IMG4198_3.jpg', 'IMG3077_3.jpg', 'IMG3232_4.jpg', 'IMG2914_2.jpg', 'IMG3845_5.jpg', 'IMG3368_2.jpg', 'IMG3334_2.jpg', 'IMG3129_5.jpg', 'IMG3323_2.jpg', 'IMG2924_4.jpg', 'IMG3784_5.jpg', 'IMG3334_2.jpg', 'IMG2772_2.jpg', 'IMG3251_3.jpg', 'IMG3725_1.jpg', 'IMG3247_2.jpg', 'IMG3323_2.jpg', 'IMG3323_2.jpg', 'IMG2768_2.jpg', 'IMG3794_2.jpg', 'IMG3709_3.jpg', 'IMG3334_2.jpg', 'IMG3036_5.jpg', 'IMG3368_2.jpg', 'IMG2768_2.jpg', 'IMG3597_2.jpg', 'IMG2923_4.jpg', 'IMG3334_2.jpg', 'IMG3223_3.jpg', 'IMG3418_3.jpg', 'IMG3149_3.jpg', 'IMG3323_2.jpg', 'IMG3593_1.jpg', 'IMG3934_1.jpg', 'IMG2867_2.jpg', 'IMG3232_4.jpg', 'IMG3048_1.jpg', 'IMG3036_5.jpg', 'IMG3323_2.jpg', 'IMG3953_2.jpg', 'IMG3794_2.jpg', 'IMG2928_5.jpg', 'IMG3323_2.jpg', 'IMG2943_2.jpg', 'IMG3575_3.jpg', 'IMG2999_3.jpg', 'IMG3368_2.jpg', 'IMG3738_3.jpg', 'IMG4188_1.jpg', 'IMG3334_2.jpg', 'IMG3248_1.jpg', 'IMG3413_2.jpg', 'IMG3283_4.jpg', 'IMG3762_5.jpg', 'IMG3189_3.jpg', 'IMG3709_3.jpg', 'IMG3323_2.jpg', 'IMG3334_2.jpg', 'IMG3645_4.jpg', 'IMG3582_4.jpg', 'IMG2923_4.jpg', 'IMG3404_2.jpg', 'IMG3334_2.jpg', 'IMG3323_2.jpg', 'IMG2792_3.jpg', 'IMG3218_3.jpg', 'IMG2792_3.jpg', 'IMG3323_2.jpg', 'IMG3149_3.jpg', 'IMG4149_5.jpg', 'IMG4117_2.jpg', 'IMG3036_5.jpg', 'IMG3334_2.jpg', 'IMG3415_1.jpg', 'IMG4117_2.jpg', 'IMG3323_2.jpg', 'IMG2817_5.jpg', 'IMG3645_4.jpg', 'IMG2845_4.jpg', 'IMG3334_2.jpg', 'IMG3992_2.jpg', 'IMG3323_2.jpg', 'IMG4186_4.jpg', 'IMG2802_1.jpg', 'IMG3323_2.jpg', 'IMG3502_2.jpg', 'IMG3828_2.jpg', 'IMG3415_1.jpg', 'IMG3854_1.jpg', 'IMG4014_5.jpg', 'IMG3368_2.jpg', 'IMG3286_4.jpg', 'IMG4188_1.jpg', 'IMG3323_2.jpg', 'IMG3727_2.jpg', 'IMG3076_2.jpg', 'IMG3334_2.jpg', 'IMG3323_2.jpg', 'IMG3721_4.jpg', 'IMG3323_2.jpg', 'IMG3334_2.jpg', 'IMG3323_2.jpg', 'IMG3334_2.jpg', 'IMG3232_4.jpg', 'IMG3088_2.jpg', 'IMG3721_4.jpg', 'IMG3243_1.jpg', 'IMG2787_3.jpg', 'IMG3934_1.jpg', 'IMG3323_2.jpg', 'IMG3899_1.jpg', 'IMG3500_2.jpg', 'IMG3232_4.jpg', 'IMG3323_2.jpg', 'IMG3502_2.jpg', 'IMG3681_5.jpg', 'IMG3999_3.jpg', 'IMG3323_2.jpg', 'IMG3864_3.jpg', 'IMG2867_2.jpg', 'IMG3822_1.jpg', 'IMG2927_3.jpg', 'IMG3794_2.jpg', 'IMG3420_1.jpg', 'IMG3848_3.jpg', 'IMG3232_4.jpg', 'IMG3323_2.jpg', 'IMG2914_2.jpg', 'IMG4212_3.jpg', 'IMG3323_2.jpg', 'IMG4168_1.jpg', 'IMG2902_1.jpg', 'IMG3234_1.jpg', 'IMG3323_2.jpg', 'IMG3794_2.jpg', 'IMG3247_2.jpg', 'IMG3283_4.jpg', 'IMG3415_1.jpg', 'IMG2867_2.jpg', 'IMG3380_2.jpg', 'IMG4150_2.jpg', 'IMG3662_2.jpg', 'IMG3323_2.jpg', 'IMG4126_4.jpg', 'IMG3762_5.jpg', 'IMG2914_2.jpg', 'IMG3036_5.jpg', 'IMG2746_2.jpg', 'IMG2792_3.jpg', 'IMG2768_2.jpg', 'IMG3171_2.jpg', 'IMG3334_2.jpg', 'IMG3323_2.jpg', 'IMG3323_2.jpg', 'IMG3323_2.jpg', 'IMG3549_4.jpg', 'IMG3183_4.jpg', 'IMG3334_2.jpg', 'IMG3036_5.jpg', 'IMG2792_3.jpg', 'IMG3323_2.jpg', 'IMG3323_2.jpg', 'IMG2919_4.jpg', 'IMG3762_5.jpg', 'IMG3413_2.jpg', 'IMG3129_5.jpg', 'IMG3323_2.jpg', 'IMG3762_5.jpg', 'IMG4126_4.jpg', 'IMG2873_3.jpg', 'IMG3582_4.jpg', 'IMG2800_1.jpg', 'IMG3681_5.jpg', 'IMG3383_1.jpg', 'IMG3334_2.jpg', 'IMG4090_1.jpg', 'IMG3334_2.jpg', 'IMG3283_4.jpg', 'IMG4194_2.jpg', 'IMG3323_2.jpg', 'IMG3461_2.jpg', 'IMG2867_2.jpg', 'IMG3845_5.jpg', 'IMG3334_2.jpg', 'IMG3036_5.jpg', 'IMG4053_1.jpg', 'IMG4001_4.jpg', 'IMG3794_2.jpg', 'IMG3848_3.jpg', 'IMG3575_3.jpg', 'IMG3232_4.jpg', 'IMG2863_2.jpg', 'IMG3232_4.jpg', 'IMG3036_5.jpg', 'IMG3057_3.jpg', 'IMG3323_2.jpg', 'IMG3762_5.jpg', 'IMG3232_4.jpg', 'IMG4153_1.jpg', 'IMG3323_2.jpg', 'IMG2929_1.jpg', 'IMG4230_4.jpg', 'IMG3323_2.jpg', 'IMG3189_3.jpg', 'IMG3845_5.jpg', 'IMG3794_2.jpg', 'IMG3490_1.jpg', 'IMG3323_2.jpg', 'IMG4013_1.jpg', 'IMG2792_3.jpg', 'IMG3500_2.jpg', 'IMG3323_2.jpg', 'IMG4145_4.jpg', 'IMG3232_4.jpg', 'IMG3247_2.jpg', 'IMG3378_3.jpg', 'IMG3880_4.jpg', 'IMG3603_5.jpg', 'IMG3794_2.jpg', 'IMG4025_1.jpg', 'IMG3232_4.jpg', 'IMG3768_4.jpg', 'IMG2782_5.jpg', 'IMG3366_1.jpg', 'IMG3582_4.jpg', 'IMG3287_5.jpg', 'IMG3368_2.jpg', 'IMG3323_2.jpg', 'IMG3575_3.jpg', 'IMG3113_1.jpg', 'IMG3200_4.jpg', 'IMG2879_5.jpg', 'IMG2772_2.jpg', 'IMG3950_4.jpg', 'IMG3323_2.jpg', 'IMG3992_2.jpg', 'IMG3911_4.jpg', 'IMG3323_2.jpg', 'IMG3323_2.jpg', 'IMG3500_2.jpg', 'IMG2999_3.jpg', 'IMG3334_2.jpg', 'IMG3036_5.jpg', 'IMG4090_1.jpg', 'IMG3597_2.jpg', 'IMG3323_2.jpg', 'IMG4141_4.jpg', 'IMG3070_2.jpg', 'IMG3171_2.jpg', 'IMG2952_4.jpg', 'IMG3593_1.jpg', 'IMG3415_1.jpg', 'IMG2888_2.jpg', 'IMG3575_3.jpg', 'IMG3382_2.jpg', 'IMG3420_1.jpg', 'IMG3334_2.jpg', 'IMG3323_2.jpg', 'IMG3603_5.jpg', 'IMG3323_2.jpg', 'IMG3992_2.jpg', 'IMG3762_5.jpg']

# Image Segmentation

In [ ]:
# Codes from Tutorial

In [ ]:
import pixellib

In [ ]:
from pixellib.semantic import semantic_segmentation

# Created an instance of semantic segmentation class
segment_image = semantic_segmentation()

# Call the function to load the xception model trained on pascal voc.
segment_image.load_pascalvoc_model("deeplabv3_xception_tf_dim_ordering_tf_kernels.h5") 

# Performs segmentation on an image and the segmentation is done in the pascalvoc's color format.
segment_image.segmentAsPascalvoc("sample4.jpg", output_image_name = "output.jpg")
# Obtain an image with segmentation overlay on the objects by setting overlay = True
segment_image.segmentAsPascalvoc("sample4.jpg", output_image_name = "overlap.jpg", overlay = True)


# Display the results
# Read an image into BGR Format and Convert to RGB
img = cv2.cvtColor(cv2.imread('sample4.jpg'),cv2.COLOR_BGR2RGB)
img1 = cv2.cvtColor(cv2.imread('output.jpg'),cv2.COLOR_BGR2RGB)
img2 = cv2.cvtColor(cv2.imread('overlap.jpg'),cv2.COLOR_BGR2RGB)
# Plot them out
plt.figure(figsize=(10,10))
plt.subplot(3,1,1),plt.imshow(img)
plt.title("Original Image"), plt.axis('off')
plt.subplot(3,1,2),plt.imshow(img1)
plt.title('Semantic Segmentation Result'), plt.axis('off')
plt.subplot(3,1,3),plt.imshow(img2)
plt.title('Image with Segmentation Overlay'), plt.axis('off')
plt.show()

# Average Hash

In [ ]:
# code from: https://pypi.org/project/ImageHash/

In [ ]:
from PIL import Image
import imagehash
import os
def compare2(train_data_name_list,test_data_name_list):
  result_name_list = []
  for test_data_name in test_data_name_list:
    k=0
    for train_data_name in train_data_name_list:
      #use the build in function average_hash
      a_hash = imagehash.average_hash(Image.open('./train'+'/'+train_data_name))
      b_hash = imagehash.average_hash(Image.open('./test'+'/'+test_data_name))
      if abs(a_hash - b_hash) > k:
        k = abs(a_hash - b_hash)
        current_name = train_data_name
      else:
        continue
    print(current_name)
    result_name_list.append(current_name)
  print('result_name:',result_name_list)
  return result_name_list

# print(a_hash - b_hash)
train_data_name_list = os.listdir('./train/')#[:3000]
test_data_name_list = os.listdir('./test/')#[:20]
compare2(train_data_name_list,test_data_name_list)

# Perceptron Hash

In [ ]:
from PIL import Image
import imagehash
import os

def compare2(train_data_name_list,test_data_name_list):
  result_name_list = []
  for test_data_name in test_data_name_list:
    k=0
    for train_data_name in train_data_name_list:
      hash1 = imagehash.phash(Image.open('./train'+'/'+train_data_name))
      hash2 = imagehash.phash(Image.open('./test'+'/'+test_data_name))
      if 1 - (hash1 - hash2)/len(hash1.hash)**2 > k:
        k = 1 - (hash1 - hash2)/len(hash1.hash)**2
        current_name = train_data_name
      else:
        continue
    print(current_name)
    result_name_list.append(current_name)
  print('result_name:',result_name_list)
  return result_name_list

train_data_name_list = os.listdir('./train/')#[:3000]
test_data_name_list = os.listdir('./test/')#[:20]
compare2(train_data_name_list,test_data_name_list)

# Compute similarity for the feature matching method

In [ ]:
import cv2
import numpy as np
import os
from operator import itemgetter 
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.applications.vgg19 import VGG19

# 1. prepare image sets
## (1).load labelled images
label_img_fnames = list(os.listdir(label_img_dir)) # the grader should define the list by self.
train_img_names = [fname.split('.')[0] for fname in label_img_fnames]
train_imgs = []
for names in train_img_names:
    img = cv2.imread(os.path.join(label_img_dir, f'{names}.jpg'))
    img = cv2.resize(img, (w, h), interpolation = cv2.INTER_AREA)
    train_imgs.append(img)
train_imgs = np.array(train_imgs)


## (2).load unlabelled images 
unlabel_img_fnames = list(os.listdir(unlabel_img_dir)) # the grader should define the list by self.
pred_img_names = [fname.split('.')[0] for fname in unlabel_img_fnames]
pred_imgs = []
for names in pred_img_names:
    img = cv2.imread(os.path.join(unlabel_img_dir, f'{names}.jpg'))
    img = cv2.resize(img, (w, h), interpolation = cv2.INTER_AREA)
    pred_imgs.append(img)
pred_imgs = np.array(pred_imgs)


# 2.prepare the feature extractor
X_train = train_imgs
X_pred = pred_imgs

img_shape = list(X_train[0].shape[1:])  # the shape of input image. 
base_model = VGG19(weights='imagenet', input_shape=img_shape, include_top=False)
fea_model= tf.keras.Model(base_model.inputs, base_model.layers[-1].output)

ss_train_fea = fea_model.predict(X_train)
ss_pred_fea = fea_model.predict(X_pred)

# compute the mean square error
# the output shape will be : 1200 x 7500
res = []
for i, pred_img in enumerate(ss_pred_fea):
    A = pred_img.flatten()
    mses = []
    for j, train_img in enumerate(ss_train_fea):
        B = train_img.flatten()
        mse = np.square(A-B).mean(axis=None)
        mses.append(mse)
    res.append(mses)


# sort and select the 10 most similar objects
# the shape of ans will be 1200 x 10. 
# key is file name for a unlabelled image, 
# value is a list of candidate of 10 images from the labelled image set.
ans = dict()
rank = 10
for idx, row in enumerate(res):
    indices = sorted(range(len(row)), key=row.__getitem__)[:rank]
    ans[pred_img_names[idx]] = [fname for fname in itemgetter(*indices)(train_img_names)]


In [ ]:
import pandas as pd
import ast
# Convert dictonary to Dataframe
ans = pd.DataFrame.from_dict(ans,orient='index')


In [ ]:
def compare(gray,scene_gray):
  sift = cv2.SIFT_create() # if cv2 version >= 4.4.0 
  # sift = cv2.xfeatures2d.SIFT_create() # if cv2 version = 4.3.x 
  
  # Compute SIFT keypoints and descriptors
  kp1, des1 = sift.detectAndCompute(gray,None)
  kp2, des2 = sift.detectAndCompute(scene_gray,None)

  # if kp1 == [] or len(kp1)<5 or kp2 == [] or len(kp2) < 5:
    # return 0

  FLANN_INDEX_KDTREE = 1
  index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
  search_params = dict(checks=50)   # or pass empty dictionary
  flann = cv2.FlannBasedMatcher(index_params,search_params)

  # Matching descriptor using KNN algorithm
  matches = flann.knnMatch(des1,des2,k=2)

  # Create a mask to draw all good matches
  matchesMask = []

  # Store all good matches as per Lowe's Ratio test.
  # good = []
  good=0
  for m,n in matches:
    if m.distance < 0.7*n.distance:
        # good.append(m)
        good = good + 1
        matchesMask.append([1,0]) # Match
    else:
        matchesMask.append([0,0]) # Mismatch
       
        

  return (good/len(matchesMask))

In [ ]:
# output_csv = pd.read_csv('./output.csv')

test_csv = ans

test_data_name_list = test_csv['pred_img_name'].values.tolist()

result_name = []

for j in test_data_name_list:
    k = 0
    train_data_name_list = ast.literal_eval(test_csv.loc[test_csv['pred_img_name'] == j]['train_img_names'].tolist()[0])
    #print('train_data_name_list:', train_data_name_list)
    for i in train_data_name_list:
        try:
            current_result=compare('/home/kali/Documents/Computer Vision/Project/train/' + i + '.jpg', '/home/kali/Documents/Computer Vision/Project/test/' + j + '.jpg')
        except:
            continue
        if current_result > k:
            k = current_result
            current_name = i
        else:
            continue
    if k == 0:
        current_name = choice(train_data_name_list)
    print(current_name)
    result_name.append(current_name)


print('result_name:',result_name)

In [ ]:
# VGG19+FeatureMatching
# The result of the computation
#result_names = ['IMG4144_3', 'IMG3250_4', 'IMG3824_2', 'IMG3205_2', 'IMG3699_2', 'IMG3544_5', 'IMG4003_2', 'IMG4178_2', 'IMG3780_5', 'IMG3066_3', 'IMG4212_3', 'IMG4064_2', 'IMG2814_1', 'IMG3949_5', 'IMG2807_4', 'IMG3271_3', 'IMG4228_3', 'IMG3537_4', 'IMG3117_3', 'IMG3817_1', 'IMG4223_5', 'IMG3227_3', 'IMG2819_5', 'IMG4042_1', 'IMG3713_3', 'IMG3799_2', 'IMG2895_1', 'IMG3168_5', 'IMG2922_4', 'IMG4020_5', 'IMG2909_4', 'IMG3018_5', 'IMG3920_5', 'IMG4179_5', 'IMG2839_1', 'IMG3243_1', 'IMG3016_3', 'IMG2781_5', 'IMG2814_1', 'IMG2903_4', 'IMG3012_3', 'IMG2951_5', 'IMG4137_1', 'IMG3323_1', 'IMG3791_4', 'IMG3581_5', 'IMG3333_1', 'IMG3865_3', 'IMG2985_5', 'IMG3008_3', 'IMG3353_2', 'IMG3071_1', 'IMG3823_3', 'IMG4242_1', 'IMG2882_5', 'IMG3330_2', 'IMG2805_2', 'IMG2752_1', 'IMG3070_3', 'IMG3219_4', 'IMG4041_5', 'IMG3219_4', 'IMG4093_3', 'IMG3177_5', 'IMG4157_1', 'IMG3841_2', 'IMG3128_3', 'IMG3599_5', 'IMG2914_1', 'IMG3002_4', 'IMG2966_5', 'IMG3832_5', 'IMG3936_3', 'IMG3756_3', 'IMG3237_5', 'IMG3471_5', 'IMG3932_1', 'IMG4206_3', 'IMG3949_2', 'IMG3632_5', 'IMG3036_3', 'IMG3671_5', 'IMG3782_4', 'IMG2782_3', 'IMG3298_1', 'IMG3719_4', 'IMG2902_1', 'IMG3015_1', 'IMG3083_2', 'IMG3931_3', 'IMG3651_5', 'IMG3243_1', 'IMG3104_1', 'IMG3857_1', 'IMG3188_2', 'IMG3000_4', 'IMG3969_4', 'IMG3707_1', 'IMG3932_2', 'IMG3732_5', 'IMG3154_2', 'IMG3134_5', 'IMG3168_3', 'IMG2757_5', 'IMG3913_5', 'IMG3623_5', 'IMG2832_1', 'IMG3108_1', 'IMG3582_2', 'IMG2940_4', 'IMG3515_2', 'IMG3172_4', 'IMG2943_5', 'IMG3961_1', 'IMG4143_2', 'IMG3219_4', 'IMG4211_2', 'IMG4135_1', 'IMG3053_4', 'IMG3250_4', 'IMG3217_1', 'IMG3108_2', 'IMG3759_5', 'IMG3873_3', 'IMG2889_5', 'IMG3184_5', 'IMG3969_2', 'IMG4151_1', 'IMG4178_2', 'IMG4159_2', 'IMG3732_5', 'IMG3237_1', 'IMG3143_1', 'IMG3729_5', 'IMG3433_3', 'IMG2992_2', 'IMG2752_3', 'IMG3237_1', 'IMG4108_5', 'IMG3786_1', 'IMG3525_4', 'IMG2953_5', 'IMG2873_4', 'IMG3374_3', 'IMG3893_2', 'IMG4032_4', 'IMG3002_3', 'IMG3887_1', 'IMG3079_3', 'IMG4041_5', 'IMG4155_3', 'IMG2861_1', 'IMG3373_4', 'IMG3099_3', 'IMG2948_4', 'IMG2760_2', 'IMG2792_3', 'IMG4028_3', 'IMG4185_4', 'IMG3307_2', 'IMG3706_2', 'IMG2762_3', 'IMG3131_1', 'IMG3889_3', 'IMG2745_4', 'IMG3818_1', 'IMG2819_4', 'IMG2932_5', 'IMG3612_4', 'IMG4185_4', 'IMG3468_2', 'IMG3292_1', 'IMG3108_1', 'IMG3711_3', 'IMG3431_1', 'IMG4151_2', 'IMG3860_3', 'IMG2878_3', 'IMG4156_2', 'IMG2974_1', 'IMG3754_5', 'IMG2941_2', 'IMG3190_3', 'IMG3031_3', 'IMG3713_3', 'IMG3201_2', 'IMG3866_1', 'IMG3166_3', 'IMG4116_1', 'IMG4125_4', 'IMG4074_4', 'IMG3248_1', 'IMG2940_3', 'IMG4083_4', 'IMG2941_2', 'IMG3663_2', 'IMG2987_3', 'IMG4242_2', 'IMG3777_4', 'IMG3865_3', 'IMG3333_1', 'IMG4155_3', 'IMG2932_4', 'IMG2820_2', 'IMG4178_2', 'IMG4017_2', 'IMG2851_5', 'IMG2842_2', 'IMG4116_1', 'IMG2814_1', 'IMG4001_2', 'IMG3257_3', 'IMG3869_4', 'IMG3004_1', 'IMG4011_4', 'IMG2878_2', 'IMG3651_5', 'IMG3674_3', 'IMG3867_2', 'IMG3580_3', 'IMG3411_2', 'IMG3816_5', 'IMG2989_4', 'IMG3230_1', 'IMG3051_1', 'IMG2863_3', 'IMG4147_3', 'IMG2774_1', 'IMG3903_3', 'IMG3047_1', 'IMG3207_3', 'IMG4099_4', 'IMG3172_2', 'IMG4234_1', 'IMG2925_2', 'IMG3065_2', 'IMG2889_5', 'IMG2970_1', 'IMG2857_1', 'IMG3545_3', 'IMG4104_5', 'IMG3861_3', 'IMG2985_5', 'IMG4101_2', 'IMG3572_1', 'IMG3999_5', 'IMG4042_5', 'IMG3034_1', 'IMG4159_3', 'IMG3401_4', 'IMG3291_4', 'IMG3796_3', 'IMG4074_5', 'IMG4215_4', 'IMG2816_1', 'IMG3493_4', 'IMG3165_4', 'IMG3195_2', 'IMG4162_4', 'IMG3049_2', 'IMG3162_5', 'IMG4212_3', 'IMG2746_4', 'IMG3808_2', 'IMG3666_5', 'IMG4242_2', 'IMG3771_1', 'IMG2883_4', 'IMG4061_4', 'IMG2941_5', 'IMG4178_3', 'IMG2851_2', 'IMG2895_4', 'IMG4116_1', 'IMG2944_3', 'IMG3355_2', 'IMG3184_3', 'IMG3882_2', 'IMG3798_2', 'IMG4239_1', 'IMG2805_3', 'IMG3620_4', 'IMG4123_4', 'IMG4006_2', 'IMG2786_4', 'IMG3664_4', 'IMG3190_3', 'IMG3561_2', 'IMG3112_2', 'IMG3723_2', 'IMG3184_2', 'IMG2827_4', 'IMG3318_1', 'IMG3865_3', 'IMG3676_2', 'IMG3409_1', 'IMG3071_2', 'IMG2993_2', 'IMG3241_4', 'IMG3782_4', 'IMG2769_5', 'IMG3839_3', 'IMG3910_3', 'IMG2991_4', 'IMG4024_5', 'IMG4062_4', 'IMG2869_1', 'IMG4172_1', 'IMG3969_2', 'IMG2812_1', 'IMG4026_2', 'IMG3681_1', 'IMG2920_3', 'IMG2921_4', 'IMG3478_2', 'IMG2905_2', 'IMG3237_5', 'IMG3085_4', 'IMG2812_1', 'IMG2966_4', 'IMG4157_1', 'IMG4212_2', 'IMG3960_3', 'IMG2894_3', 'IMG3799_3', 'IMG2795_5', 'IMG3059_3', 'IMG3084_3', 'IMG3727_3', 'IMG4200_2', 'IMG3231_5', 'IMG3193_4', 'IMG3837_5', 'IMG4074_5', 'IMG2807_5', 'IMG3696_5', 'IMG3367_4', 'IMG3024_1', 'IMG3969_2', 'IMG4225_2', 'IMG3071_3', 'IMG3271_1', 'IMG3037_1', 'IMG3544_4', 'IMG4035_2', 'IMG2989_4', 'IMG2832_5', 'IMG4181_2', 'IMG3620_4', 'IMG2897_5', 'IMG3226_2', 'IMG3469_5', 'IMG3342_3', 'IMG3347_2', 'IMG3231_3', 'IMG3936_4', 'IMG3169_3', 'IMG3568_5', 'IMG3229_2', 'IMG2782_3', 'IMG3809_1', 'IMG4154_1', 'IMG3776_2', 'IMG3102_1', 'IMG3868_4', 'IMG3879_3', 'IMG2874_2', 'IMG3191_2', 'IMG3261_4', 'IMG4067_5', 'IMG3242_1', 'IMG4196_5', 'IMG4174_5', 'IMG3936_1', 'IMG3026_3', 'IMG2931_4', 'IMG3096_5', 'IMG3912_5', 'IMG3772_5', 'IMG3668_1', 'IMG3088_5', 'IMG3241_4', 'IMG3323_1', 'IMG3561_2', 'IMG4162_4', 'IMG4242_2', 'IMG3782_4', 'IMG3489_5', 'IMG3691_5', 'IMG3202_2', 'IMG3620_4', 'IMG3666_2', 'IMG3197_2', 'IMG3839_4', 'IMG3971_1', 'IMG2991_4', 'IMG4070_2', 'IMG3049_2', 'IMG4038_5', 'IMG3544_4', 'IMG3975_3', 'IMG3190_2', 'IMG2832_4', 'IMG2840_1', 'IMG4066_4', 'IMG4028_3', 'IMG3984_4', 'IMG3096_1', 'IMG2940_4', 'IMG3346_4', 'IMG3023_5', 'IMG3744_4', 'IMG2943_1', 'IMG3089_5', 'IMG3818_2', 'IMG3052_5', 'IMG3292_5', 'IMG4062_5', 'IMG3894_1', 'IMG3122_1', 'IMG4185_4', 'IMG3651_5', 'IMG3242_5', 'IMG2753_4', 'IMG3929_2', 'IMG4207_1', 'IMG4071_4', 'IMG4094_3', 'IMG3984_5', 'IMG3054_2', 'IMG3210_4', 'IMG2940_3', 'IMG4204_1', 'IMG2880_1', 'IMG3897_2', 'IMG3345_4', 'IMG2802_1', 'IMG2786_2', 'IMG3197_2', 'IMG2932_4', 'IMG3691_5', 'IMG3611_1', 'IMG4054_2', 'IMG3868_1', 'IMG2860_4', 'IMG3346_4', 'IMG3284_2', 'IMG3861_3', 'IMG3427_5', 'IMG2940_3', 'IMG2882_3', 'IMG3280_4', 'IMG3083_1', 'IMG3805_2', 'IMG3820_3', 'IMG4238_5', 'IMG3881_1', 'IMG3165_3', 'IMG3713_3', 'IMG4187_1', 'IMG3286_1', 'IMG3291_5', 'IMG2855_1', 'IMG3763_2', 'IMG3707_1', 'IMG3908_5', 'IMG3580_3', 'IMG4242_1', 'IMG3155_2', 'IMG3693_3', 'IMG3257_3', 'IMG3369_4', 'IMG4208_1', 'IMG3955_2', 'IMG3790_5', 'IMG3505_1', 'IMG4089_1', 'IMG3759_5', 'IMG3622_3', 'IMG3955_2', 'IMG3318_1', 'IMG2940_3', 'IMG3067_1', 'IMG3722_2', 'IMG3130_1', 'IMG3476_2', 'IMG3242_1', 'IMG4150_3', 'IMG3987_2', 'IMG2834_4', 'IMG4144_3', 'IMG2911_2', 'IMG3713_3', 'IMG3148_5', 'IMG3734_2', 'IMG3123_4', 'IMG3854_3', 'IMG4188_1', 'IMG4188_1', 'IMG4020_5', 'IMG3006_2', 'IMG2887_1', 'IMG3270_3', 'IMG3499_3', 'IMG2987_3', 'IMG3193_4', 'IMG3520_4', 'IMG3160_4', 'IMG2995_1', 'IMG2974_1', 'IMG4202_3', 'IMG3088_1', 'IMG3037_5', 'IMG4162_1', 'IMG3083_2', 'IMG2769_5', 'IMG3154_2', 'IMG2854_4', 'IMG3108_2', 'IMG2832_4', 'IMG2940_4', 'IMG3805_5', 'IMG3200_5', 'IMG3389_4', 'IMG4027_4', 'IMG3732_5', 'IMG4004_2', 'IMG2865_3', 'IMG2979_5', 'IMG3133_1', 'IMG3432_3', 'IMG2940_4', 'IMG3117_1', 'IMG3984_5', 'IMG3054_5', 'IMG3496_4', 'IMG4053_3', 'IMG3106_2', 'IMG3134_4', 'IMG3250_4', 'IMG3694_1', 'IMG3274_3', 'IMG3883_1', 'IMG3699_4', 'IMG3093_2', 'IMG3789_4', 'IMG3114_5', 'IMG2845_4', 'IMG3669_5', 'IMG4157_1', 'IMG3946_4', 'IMG3020_4', 'IMG4060_1', 'IMG2940_4', 'IMG3490_4', 'IMG3495_4', 'IMG3347_2', 'IMG3361_2', 'IMG3624_1', 'IMG4099_5', 'IMG4122_3', 'IMG3347_2', 'IMG2787_2', 'IMG2761_2', 'IMG4117_1', 'IMG3508_5', 'IMG3763_2', 'IMG2940_4', 'IMG3946_4', 'IMG3693_3', 'IMG3127_2', 'IMG3026_3', 'IMG3583_5', 'IMG4174_1', 'IMG4242_1', 'IMG3129_2', 'IMG3237_5', 'IMG3379_5', 'IMG3920_3', 'IMG3987_3', 'IMG2974_1', 'IMG3254_4', 'IMG3612_4', 'IMG3956_3', 'IMG3840_1', 'IMG4239_3', 'IMG3580_3', 'IMG2845_4', 'IMG2943_4', 'IMG2800_2', 'IMG3839_3', 'IMG3117_5', 'IMG4088_3', 'IMG3839_3', 'IMG4178_2', 'IMG2841_4', 'IMG3651_4', 'IMG3000_4', 'IMG4117_3', 'IMG3049_3', 'IMG3879_3', 'IMG3916_2', 'IMG3174_3', 'IMG2845_5', 'IMG3640_3', 'IMG3225_5', 'IMG3154_2', 'IMG2749_4', 'IMG3948_2', 'IMG4130_4', 'IMG4180_4', 'IMG2844_3', 'IMG4164_1', 'IMG4131_4', 'IMG3839_5', 'IMG2955_1', 'IMG4178_2', 'IMG2771_1', 'IMG3902_5', 'IMG2940_4', 'IMG3824_3', 'IMG3930_5', 'IMG4206_5', 'IMG3691_3', 'IMG3886_5', 'IMG4157_1', 'IMG2832_4', 'IMG3622_3', 'IMG4242_2', 'IMG3919_2', 'IMG3799_4', 'IMG4229_5', 'IMG3432_5', 'IMG2986_1', 'IMG3537_5', 'IMG4116_1', 'IMG3860_3', 'IMG3868_3', 'IMG2786_2', 'IMG4017_5', 'IMG2816_3', 'IMG4062_4', 'IMG3782_4', 'IMG2829_3', 'IMG3868_1', 'IMG4116_1', 'IMG3235_3', 'IMG3499_4', 'IMG2788_5', 'IMG3749_2', 'IMG3433_1', 'IMG3763_1', 'IMG3857_1', 'IMG3884_1', 'IMG4100_3', 'IMG4183_2', 'IMG2894_5', 'IMG3685_1', 'IMG3035_2', 'IMG4078_2', 'IMG4002_2', 'IMG3911_4', 'IMG3939_5', 'IMG3713_3', 'IMG4105_5', 'IMG2932_5', 'IMG2940_3', 'IMG3224_2', 'IMG3612_4', 'IMG3840_2', 'IMG3012_5', 'IMG3865_3', 'IMG2757_3', 'IMG3931_1', 'IMG3333_5', 'IMG4120_2', 'IMG4156_2', 'IMG4242_1', 'IMG3911_2', 'IMG2786_5', 'IMG3941_3', 'IMG3761_4', 'IMG3219_4', 'IMG3897_2', 'IMG3913_5', 'IMG3890_3', 'IMG2941_1', 'IMG2940_3', 'IMG3535_1', 'IMG3242_5', 'IMG3713_3', 'IMG2956_4', 'IMG2982_4', 'IMG3081_4', 'IMG3517_2', 'IMG3696_5', 'IMG2953_2', 'IMG4117_4', 'IMG3812_2', 'IMG4221_2', 'IMG3229_2', 'IMG2989_3', 'IMG3237_5', 'IMG2840_5', 'IMG4239_3', 'IMG4184_5', 'IMG3237_5', 'IMG4070_5', 'IMG2962_3', 'IMG3076_5', 'IMG3389_4', 'IMG4238_2', 'IMG2974_1', 'IMG2950_4', 'IMG4055_4', 'IMG3748_1', 'IMG3155_2', 'IMG3751_1', 'IMG3085_4', 'IMG4089_3', 'IMG2832_4', 'IMG4122_3', 'IMG2877_1', 'IMG2842_1', 'IMG3291_5', 'IMG3486_4', 'IMG3237_5', 'IMG4113_4', 'IMG4090_2', 'IMG3882_5', 'IMG3210_1', 'IMG4148_5', 'IMG4187_5', 'IMG3347_2', 'IMG3174_3', 'IMG4056_3', 'IMG3393_1', 'IMG2895_4', 'IMG3868_1', 'IMG3181_3', 'IMG4185_5', 'IMG3905_3', 'IMG4122_3', 'IMG3371_5', 'IMG3219_4', 'IMG4206_5', 'IMG3038_4', 'IMG3648_1', 'IMG3702_2', 'IMG3172_2', 'IMG3042_5', 'IMG4242_1', 'IMG3951_1', 'IMG4026_2', 'IMG3545_3', 'IMG4197_5', 'IMG4155_3', 'IMG2815_4', 'IMG3250_4', 'IMG3012_2', 'IMG2759_3', 'IMG3177_2', 'IMG3997_1', 'IMG3241_4', 'IMG3687_1', 'IMG4069_4', 'IMG2925_2', 'IMG3333_1', 'IMG3979_2', 'IMG3219_4', 'IMG4062_4', 'IMG3732_5', 'IMG2769_2', 'IMG3612_4', 'IMG3342_3', 'IMG3051_1', 'IMG3125_4', 'IMG3948_2', 'IMG2833_3', 'IMG3241_2', 'IMG3205_5', 'IMG3478_2', 'IMG4089_5', 'IMG2810_5', 'IMG3101_5', 'IMG3499_1', 'IMG2755_4', 'IMG3362_2', 'IMG2869_3', 'IMG3205_2', 'IMG3190_3', 'IMG3747_4', 'IMG3069_5', 'IMG3133_3', 'IMG4052_5', 'IMG4230_5', 'IMG2912_3', 'IMG3355_1', 'IMG3188_5', 'IMG3327_1', 'IMG2922_4', 'IMG3791_4', 'IMG3229_2', 'IMG3561_2', 'IMG3451_3', 'IMG4038_2', 'IMG3219_3', 'IMG3697_2', 'IMG3389_5', 'IMG4056_3', 'IMG3739_2', 'IMG3795_1', 'IMG4239_4', 'IMG3071_2', 'IMG4168_5', 'IMG3887_1', 'IMG3879_4', 'IMG4100_1', 'IMG2965_3', 'IMG3436_5', 'IMG3055_1', 'IMG4122_3', 'IMG2974_1', 'IMG4163_4', 'IMG3396_5', 'IMG3460_2', 'IMG2960_2', 'IMG2837_5', 'IMG3031_2', 'IMG3860_3', 'IMG4168_4', 'IMG4039_4', 'IMG2810_1', 'IMG3753_5', 'IMG4042_5', 'IMG3675_5', 'IMG2929_5', 'IMG2748_5', 'IMG4185_4', 'IMG2914_2', 'IMG3744_5', 'IMG2790_5', 'IMG3213_1', 'IMG3605_1', 'IMG2845_5', 'IMG3199_2', 'IMG3605_1', 'IMG2825_1', 'IMG3508_5', 'IMG3201_3', 'IMG3723_3', 'IMG4123_2', 'IMG3882_5', 'IMG3620_4', 'IMG3707_1', 'IMG2816_4', 'IMG2976_2', 'IMG3687_5', 'IMG2814_1', 'IMG3865_3', 'IMG3742_4', 'IMG4187_5', 'IMG3713_3', 'IMG3143_1', 'IMG3741_3', 'IMG3902_1', 'IMG3167_3', 'IMG3544_4', 'IMG3056_2', 'IMG2987_1', 'IMG2940_3', 'IMG3217_1', 'IMG2971_3', 'IMG2893_4', 'IMG4123_5', 'IMG3936_1', 'IMG3743_5', 'IMG3713_3', 'IMG2954_4', 'IMG4112_4', 'IMG3508_1', 'IMG3936_2', 'IMG3802_4', 'IMG4157_1', 'IMG4078_5', 'IMG3120_2', 'IMG4206_1', 'IMG4123_5', 'IMG3138_4', 'IMG3816_1', 'IMG3341_3', 'IMG2759_3', 'IMG3114_4', 'IMG3274_5', 'IMG2768_1', 'IMG3768_2', 'IMG4188_1', 'IMG3040_1', 'IMG2837_2', 'IMG3432_3', 'IMG3798_2', 'IMG2786_4', 'IMG3493_3', 'IMG3165_3', 'IMG4177_3', 'IMG3079_1', 'IMG3795_1', 'IMG3791_1', 'IMG4242_1', 'IMG4044_1', 'IMG3396_5', 'IMG3065_3', 'IMG4138_5', 'IMG3311_3', 'IMG4062_4', 'IMG4173_1', 'IMG3505_1', 'IMG2882_2', 'IMG4180_4', 'IMG4226_3', 'IMG3760_1', 'IMG2988_5', 'IMG4112_4', 'IMG2927_4', 'IMG3333_1', 'IMG3949_3', 'IMG4234_5', 'IMG4182_4', 'IMG2848_4', 'IMG2805_1', 'IMG3821_2', 'IMG3568_4', 'IMG3710_4', 'IMG3153_1', 'IMG3656_4', 'IMG3348_2', 'IMG2778_2', 'IMG2971_3', 'IMG3043_5', 'IMG3804_3', 'IMG3776_3', 'IMG3325_4', 'IMG3169_3', 'IMG3672_5', 'IMG3173_4', 'IMG3186_2', 'IMG2970_2', 'IMG2959_4', 'IMG3237_5', 'IMG2755_4', 'IMG3703_1', 'IMG3861_3', 'IMG4185_4', 'IMG3257_2', 'IMG3517_3', 'IMG2974_5', 'IMG2889_1', 'IMG3342_3', 'IMG4055_2', 'IMG4061_3', 'IMG3186_2', 'IMG3975_1', 'IMG3663_1', 'IMG3469_2', 'IMG3220_4', 'IMG3073_4', 'IMG3792_4', 'IMG2985_3', 'IMG4011_4', 'IMG2779_1', 'IMG3651_4', 'IMG2986_1', 'IMG3882_2', 'IMG3213_5', 'IMG3260_2', 'IMG3162_4', 'IMG3222_3', 'IMG3065_3', 'IMG3966_2', 'IMG2874_3', 'IMG3060_5', 'IMG3207_4', 'IMG4175_2', 'IMG3742_4', 'IMG3984_3', 'IMG2974_1', 'IMG3245_1', 'IMG3100_4', 'IMG3006_5', 'IMG3627_1', 'IMG2884_3', 'IMG4112_4', 'IMG3612_4', 'IMG3140_1', 'IMG3882_4', 'IMG3043_4', 'IMG2995_5', 'IMG3860_3', 'IMG2831_4', 'IMG3763_1', 'IMG3861_4', 'IMG3865_3', 'IMG3025_3', 'IMG3932_5', 'IMG3237_5', 'IMG3868_3', 'IMG3916_1', 'IMG3527_5', 'IMG2940_2', 'IMG4062_4', 'IMG3237_5', 'IMG3432_2', 'IMG3239_1', 'IMG2869_2', 'IMG3079_3', 'IMG3204_4', 'IMG3839_3', 'IMG3411_2', 'IMG2971_3', 'IMG3371_5', 'IMG2922_3', 'IMG3713_3', 'IMG4133_5', 'IMG2845_5', 'IMG3699_2', 'IMG3292_1', 'IMG3318_1', 'IMG4035_1', 'IMG2940_3', 'IMG3727_3', 'IMG2895_1', 'IMG2940_4', 'IMG2971_3', 'IMG2812_1', 'IMG3298_2', 'IMG4137_1', 'IMG4169_5', 'IMG3197_3', 'IMG3651_5', 'IMG4196_1', 'IMG2988_5', 'IMG3255_5', 'IMG3975_1', 'IMG4041_5', 'IMG3237_5', 'IMG2998_1', 'IMG2814_1', 'IMG2851_1', 'IMG3757_1', 'IMG2988_3', 'IMG3620_4', 'IMG3145_5', 'IMG3375_3', 'IMG3495_4', 'IMG3006_5', 'IMG3641_3', 'IMG3795_1', 'IMG3826_5', 'IMG2911_2', 'IMG3798_2', 'IMG4050_3', 'IMG4157_1', 'IMG4116_1', 'IMG3243_3', 'IMG2770_1', 'IMG3197_5', 'IMG3912_1', 'IMG3970_5', 'IMG3953_1', 'IMG4132_3', 'IMG3038_4', 'IMG3936_3', 'IMG3170_4', 'IMG4037_3', 'IMG2838_1', 'IMG2946_3', 'IMG3755_1', 'IMG3828_1', 'IMG3877_5', 'IMG3525_4', 'IMG3232_1', 'IMG4112_3', 'IMG2954_4', 'IMG4157_1', 'IMG3994_5', 'IMG3680_3', 'IMG3752_4', 'IMG3719_3', 'IMG3115_5', 'IMG3945_2', 'IMG3700_5', 'IMG3325_5', 'IMG4090_2', 'IMG4112_4', 'IMG4099_1', 'IMG2820_1', 'IMG3836_2', 'IMG3516_5', 'IMG2833_4', 'IMG4229_2', 'IMG3930_5', 'IMG3812_1', 'IMG3499_5', 'IMG4196_1', 'IMG4039_4', 'IMG3478_2', 'IMG3976_5', 'IMG4206_2', 'IMG3796_3', 'IMG3839_3', 'IMG2956_3', 'IMG4152_3', 'IMG4146_4', 'IMG3796_1', 'IMG4242_1', 'IMG4175_5', 'IMG3181_3', 'IMG3997_1', 'IMG3932_4', 'IMG4083_1', 'IMG3984_4', 'IMG4193_3', 'IMG3925_4', 'IMG4181_2', 'IMG3743_5', 'IMG2892_3', 'IMG2882_3', 'IMG2892_2', 'IMG3946_5', 'IMG3712_5', 'IMG3061_5', 'IMG3962_2', 'IMG4117_4', 'IMG3777_4', 'IMG2967_5', 'IMG2771_5', 'IMG3920_3', 'IMG3084_1', 'IMG2825_1', 'IMG3691_4', 'IMG3190_3', 'IMG2974_1', 'IMG3456_3', 'IMG2997_5', 'IMG2815_5', 'IMG3023_2', 'IMG2803_1', 'IMG3205_3', 'IMG3656_4', 'IMG3826_4', 'IMG3071_2', 'IMG4093_2', 'IMG2991_5', 'IMG3201_1', 'IMG4185_1', 'IMG3537_5', 'IMG3025_2', 'IMG3913_5', 'IMG3752_1', 'IMG3229_2', 'IMG3025_1', 'IMG2921_4', 'IMG3027_1', 'IMG3167_5', 'IMG3241_2', 'IMG4017_5', 'IMG2954_4', 'IMG3251_4', 'IMG3271_3', 'IMG2997_5', 'IMG4185_4', 'IMG3182_2', 'IMG3009_1', 'IMG3024_5', 'IMG2777_3', 'IMG3941_3', 'IMG2932_5', 'IMG4219_3', 'IMG3665_5', 'IMG4041_1', 'IMG4093_4', 'IMG3675_5', 'IMG4056_3', 'IMG3089_5', 'IMG2796_4', 'IMG4174_2', 'IMG3237_5', 'IMG4155_3', 'IMG2786_2', 'IMG4199_2', 'IMG3190_3', 'IMG3347_1', 'IMG3962_4', 'IMG4105_3', 'IMG3122_5', 'IMG3271_3', 'IMG3312_5', 'IMG3937_1', 'IMG2813_1', 'IMG4152_2']

In [ ]:
import pandas as pd
# import pandas.dataframe as df

# This is use to read the result file from VGG19
output_csv = pd.read_csv('./output.csv')

# This is the output file of feature match which only contains the names
test_file_path = './imagenames-CGG19+FeatureMatching.csv'

# This is the train data name and coordinate csv file
train_csv = pd.read_csv('./train.csv')

# This is the original imagenames(Test names) CSV file
test_csv = pd.read_csv('./imagenames.csv')

# This function is to make the test name as a list
test_name_list = output_csv['pred_img_name'].values.tolist()

In [ ]:
with open(test_file_path, 'w', encoding='UTF8') as f:
    writer = csv.writer(f)
    writer.writerow(['id','x','y'])
    for test_name, result_name in zip(test_name_list, result_names):
      a = train_csv.loc[train_csv['id'] == result_name]
      writer.writerow([test_name,a['x'].values[0],a['y'].values[0]])